# Analyse de la structure des données

## Objectif
Avant de modéliser, comprendre :
1. **La structure** des 4 tables et leurs relations
2. **Les colonnes** disponibles et leur signification
3. **La granularité** possible du dataset (accident, véhicule, usager)
4. **Le feature engineering** pertinent pour notre problème

In [1]:
import pandas as pd
import numpy as np

# Charger les 4 tables
caract = pd.read_csv('../données/2024/caract-2024.csv', sep=';')
lieux = pd.read_csv('../données/2024/lieux-2024.csv', sep=';')
usagers = pd.read_csv('../données/2024/usagers-2024.csv', sep=';')
vehicules = pd.read_csv('../données/2024/vehicules-2024.csv', sep=';')

print("=== Dimensions ===")
print(f"caract:    {caract.shape[0]:>6} lignes, {caract.shape[1]:>2} colonnes")
print(f"lieux:     {lieux.shape[0]:>6} lignes, {lieux.shape[1]:>2} colonnes")
print(f"vehicules: {vehicules.shape[0]:>6} lignes, {vehicules.shape[1]:>2} colonnes")
print(f"usagers:   {usagers.shape[0]:>6} lignes, {usagers.shape[1]:>2} colonnes")

=== Dimensions ===
caract:     54402 lignes, 15 colonnes
lieux:      70248 lignes, 18 colonnes
vehicules:  92678 lignes, 11 colonnes
usagers:   125187 lignes, 16 colonnes


---
## 1. Comprendre les relations entre tables

Question clé : combien de lignes par accident dans chaque table ?

In [2]:
# Nombre d'accidents uniques
print("=== Accidents uniques par table ===")
print(f"caract:    {caract['Num_Acc'].nunique()} accidents")
print(f"lieux:     {lieux['Num_Acc'].nunique()} accidents")
print(f"vehicules: {vehicules['Num_Acc'].nunique()} accidents")
print(f"usagers:   {usagers['Num_Acc'].nunique()} accidents")

print("\n=== Lignes par accident (moyenne) ===")
print(f"caract:    {caract.shape[0] / caract['Num_Acc'].nunique():.1f} ligne/accident")
print(f"lieux:     {lieux.shape[0] / lieux['Num_Acc'].nunique():.1f} lignes/accident")
print(f"vehicules: {vehicules.shape[0] / vehicules['Num_Acc'].nunique():.1f} lignes/accident")
print(f"usagers:   {usagers.shape[0] / usagers['Num_Acc'].nunique():.1f} lignes/accident")

=== Accidents uniques par table ===
caract:    54402 accidents
lieux:     54402 accidents
vehicules: 54402 accidents
usagers:   54402 accidents

=== Lignes par accident (moyenne) ===
caract:    1.0 ligne/accident
lieux:     1.3 lignes/accident
vehicules: 1.7 lignes/accident
usagers:   2.3 lignes/accident


In [3]:
# Distribution du nombre de véhicules par accident
veh_par_acc = vehicules.groupby('Num_Acc').size()
print("=== Nombre de véhicules par accident ===")
print(veh_par_acc.value_counts().sort_index().head(10))

print("\n=== Nombre d'usagers par accident ===")
usa_par_acc = usagers.groupby('Num_Acc').size()
print(usa_par_acc.value_counts().sort_index().head(10))

=== Nombre de véhicules par accident ===
1     20780
2     29968
3      2909
4       574
5       124
6        30
7         6
8         6
9         3
11        2
Name: count, dtype: int64

=== Nombre d'usagers par accident ===
1      9084
2     31338
3      8162
4      3104
5      1457
6       663
7       279
8       145
9        80
10       21
Name: count, dtype: int64


### Schéma relationnel

```
CARACT (1 ligne/accident)
   │
   ├── LIEUX (1-N lignes/accident) - pourquoi plusieurs ?
   │
   └── VEHICULES (1-N lignes/accident)
           │
           └── USAGERS (1-N lignes/véhicule)
```

**Questions à se poser :**
- Pourquoi `lieux` a plusieurs lignes par accident ?
- Quelle est la clé de liaison véhicule-usager ?

In [4]:
# Explorer les clés
print("=== Colonnes de chaque table ===")
print(f"\ncaract: {caract.columns.tolist()}")
print(f"\nlieux: {lieux.columns.tolist()}")
print(f"\nvehicules: {vehicules.columns.tolist()}")
print(f"\nusagers: {usagers.columns.tolist()}")

=== Colonnes de chaque table ===

caract: ['Num_Acc', 'jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'com', 'agg', 'int', 'atm', 'col', 'adr', 'lat', 'long']

lieux: ['Num_Acc', 'catr', 'voie', 'v1', 'v2', 'circ', 'nbv', 'vosp', 'prof', 'pr', 'pr1', 'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ', 'vma']

vehicules: ['Num_Acc', 'id_vehicule', 'num_veh', 'senc', 'catv', 'obs', 'obsm', 'choc', 'manv', 'motor', 'occutc']

usagers: ['Num_Acc', 'id_usager', 'id_vehicule', 'num_veh', 'place', 'catu', 'grav', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2', 'secu3', 'locp', 'actp', 'etatp']


In [5]:
# Lien véhicule-usager
print("=== Clé véhicule dans usagers ===")
print(f"'id_vehicule' dans vehicules: {'id_vehicule' in vehicules.columns}")
print(f"'id_vehicule' dans usagers: {'id_vehicule' in usagers.columns}")
print(f"'num_veh' dans usagers: {'num_veh' in usagers.columns}")

# Vérifier la correspondance
print(f"\nExemple - 1er accident:")
premier_acc = caract['Num_Acc'].iloc[0]
print(f"Num_Acc: {premier_acc}")
print(f"Véhicules: {vehicules[vehicules['Num_Acc'] == premier_acc][['Num_Acc', 'id_vehicule', 'num_veh']]}")
print(f"Usagers: {usagers[usagers['Num_Acc'] == premier_acc][['Num_Acc', 'id_vehicule', 'num_veh', 'catu', 'grav']]}")

=== Clé véhicule dans usagers ===
'id_vehicule' dans vehicules: True
'id_vehicule' dans usagers: True
'num_veh' dans usagers: True

Exemple - 1er accident:
Num_Acc: 202400000001
Véhicules:         Num_Acc  id_vehicule num_veh
0  202400000001  155 781 758     A01
1  202400000001  155 781 759     B01
Usagers:         Num_Acc  id_vehicule num_veh  catu  grav
0  202400000001  155 781 758     A01     1     3
1  202400000001  155 781 759     B01     1     1


---
## 2. Analyse des colonnes

Pour chaque table, comprendre ce que représente chaque colonne.

In [6]:
print("=" * 60)
print("TABLE CARACT - Caractéristiques de l'accident")
print("=" * 60)
for col in caract.columns:
    print(f"\n{col}:")
    print(f"  Type: {caract[col].dtype}")
    print(f"  Valeurs uniques: {caract[col].nunique()}")
    if caract[col].nunique() <= 10:
        print(f"  Distribution: {caract[col].value_counts().to_dict()}")
    else:
        print(f"  Exemples: {caract[col].head(3).tolist()}")

TABLE CARACT - Caractéristiques de l'accident

Num_Acc:
  Type: int64
  Valeurs uniques: 54402
  Exemples: [202400000001, 202400000002, 202400000003]

jour:
  Type: int64
  Valeurs uniques: 31
  Exemples: [25, 20, 22]

mois:
  Type: int64
  Valeurs uniques: 12
  Exemples: [3, 3, 3]

an:
  Type: int64
  Valeurs uniques: 1
  Distribution: {2024: 54402}

hrmn:
  Type: str
  Valeurs uniques: 1414
  Exemples: ['07:40', '15:05', '19:30']

lum:
  Type: int64
  Valeurs uniques: 5
  Distribution: {1: 35580, 5: 8499, 3: 6101, 2: 3599, 4: 623}

dep:
  Type: str
  Valeurs uniques: 107
  Exemples: ['70', '21', '15']

com:
  Type: str
  Valeurs uniques: 11285
  Exemples: ['70285', '21054', '15012']

agg:
  Type: int64
  Valeurs uniques: 2
  Distribution: {2: 34010, 1: 20392}

int:
  Type: int64
  Valeurs uniques: 9
  Distribution: {1: 34589, 3: 6451, 2: 6289, 6: 2717, 9: 2454, 4: 1048, 7: 453, 5: 263, 8: 138}

atm:
  Type: int64
  Valeurs uniques: 9
  Distribution: {1: 41802, 2: 6779, 8: 2374, 3: 14

In [7]:
print("=" * 60)
print("TABLE LIEUX - Caractéristiques du lieu")
print("=" * 60)
for col in lieux.columns:
    print(f"\n{col}:")
    print(f"  Type: {lieux[col].dtype}")
    print(f"  Valeurs uniques: {lieux[col].nunique()}")
    if lieux[col].nunique() <= 10:
        print(f"  Distribution: {lieux[col].value_counts().to_dict()}")
    else:
        print(f"  Exemples: {lieux[col].head(3).tolist()}")

TABLE LIEUX - Caractéristiques du lieu

Num_Acc:
  Type: int64
  Valeurs uniques: 54402
  Exemples: [202400000001, 202400000002, 202400000002]

catr:
  Type: int64
  Valeurs uniques: 8
  Distribution: {4: 31977, 3: 26452, 1: 5181, 2: 3968, 7: 1941, 6: 441, 9: 225, 5: 63}

voie:
  Type: str
  Valeurs uniques: 19750
  Exemples: ['D438', "HOTEL DIEU (RUE DE L')", 'POTERNE (RUE)']

v1:
  Type: int64
  Valeurs uniques: 4
  Distribution: {0: 53866, -1: 16272, 2: 94, 3: 16}

v2:
  Type: str
  Valeurs uniques: 27
  Exemples: [nan, nan, nan]

circ:
  Type: int64
  Valeurs uniques: 5
  Distribution: {2: 43402, 1: 14195, 3: 7909, -1: 4354, 4: 388}

nbv:
  Type: str
  Valeurs uniques: 15
  Exemples: ['2', '2', '1']

vosp:
  Type: int64
  Valeurs uniques: 5
  Distribution: {0: 57329, 1: 3870, -1: 3832, 2: 2632, 3: 2585}

prof:
  Type: int64
  Valeurs uniques: 5
  Distribution: {1: 57751, 2: 10531, 3: 1026, 4: 890, -1: 50}

pr:
  Type: str
  Valeurs uniques: 461
  Exemples: ['1', ' -1', ' -1']

pr1:

In [8]:
print("=" * 60)
print("TABLE VEHICULES")
print("=" * 60)
for col in vehicules.columns:
    print(f"\n{col}:")
    print(f"  Type: {vehicules[col].dtype}")
    print(f"  Valeurs uniques: {vehicules[col].nunique()}")
    if vehicules[col].nunique() <= 15:
        print(f"  Distribution: {vehicules[col].value_counts().head(10).to_dict()}")
    else:
        print(f"  Exemples: {vehicules[col].head(3).tolist()}")

TABLE VEHICULES

Num_Acc:
  Type: int64
  Valeurs uniques: 54402
  Exemples: [202400000001, 202400000001, 202400000002]

id_vehicule:
  Type: str
  Valeurs uniques: 92678
  Exemples: ['155\xa0781\xa0758', '155\xa0781\xa0759', '155\xa0781\xa0757']

num_veh:
  Type: str
  Valeurs uniques: 45
  Exemples: ['A01', 'B01', 'A01']

senc:
  Type: int64
  Valeurs uniques: 5
  Distribution: {1: 41285, 2: 31450, 3: 15063, 0: 4812, -1: 68}

catv:
  Type: int64
  Valeurs uniques: 32
  Exemples: [7, 14, 10]

obs:
  Type: int64
  Valeurs uniques: 19
  Exemples: [0, 0, 0]

obsm:
  Type: int64
  Valeurs uniques: 8
  Distribution: {2: 64549, 0: 17572, 1: 8354, 9: 1849, 6: 138, 5: 96, 4: 90, -1: 30}

choc:
  Type: int64
  Valeurs uniques: 11
  Distribution: {1: 33858, 3: 13928, 2: 11009, 4: 8844, 8: 6307, 0: 6083, 7: 5521, 6: 3202, 5: 2558, 9: 1324}

manv:
  Type: int64
  Valeurs uniques: 28
  Exemples: [13, 21, 15]

motor:
  Type: int64
  Valeurs uniques: 8
  Distribution: {1: 72964, 3: 6000, 5: 5285, 2:

In [9]:
print("=" * 60)
print("TABLE USAGERS")
print("=" * 60)
for col in usagers.columns:
    print(f"\n{col}:")
    print(f"  Type: {usagers[col].dtype}")
    print(f"  Valeurs uniques: {usagers[col].nunique()}")
    if usagers[col].nunique() <= 15:
        print(f"  Distribution: {usagers[col].value_counts().head(10).to_dict()}")
    else:
        print(f"  Exemples: {usagers[col].head(3).tolist()}")

TABLE USAGERS

Num_Acc:
  Type: int64
  Valeurs uniques: 54402
  Exemples: [202400000001, 202400000001, 202400000002]

id_usager:
  Type: str
  Valeurs uniques: 125187
  Exemples: ['203\xa0988\xa0581', '203\xa0988\xa0582', '203\xa0988\xa0579']

id_vehicule:
  Type: str
  Valeurs uniques: 92654
  Exemples: ['155\xa0781\xa0758', '155\xa0781\xa0759', '155\xa0781\xa0757']

num_veh:
  Type: str
  Valeurs uniques: 45
  Exemples: ['A01', 'B01', 'A01']

place:
  Type: int64
  Valeurs uniques: 11
  Distribution: {1: 92567, 2: 14120, 10: 9401, 3: 2500, 4: 2419, 9: 1361, 7: 1304, 5: 703, 8: 625, 6: 184}

catu:
  Type: int64
  Valeurs uniques: 3
  Distribution: {1: 92581, 2: 23205, 3: 9401}

grav:
  Type: int64
  Valeurs uniques: 4
  Distribution: {1: 52920, 4: 49709, 3: 19126, 2: 3432}

sexe:
  Type: int64
  Valeurs uniques: 3
  Distribution: {1: 83864, 2: 38928, -1: 2395}

an_nais:
  Type: float64
  Valeurs uniques: 105
  Exemples: [2003.0, 1997.0, 1927.0]

trajet:
  Type: int64
  Valeurs unique

---
## 3. Réflexion : Quelle granularité pour le modèle ?

### Option A : 1 ligne par ACCIDENT
- **Target** : L'accident est-il mortel ? (oui/non)
- **Avantage** : Simple, correspond au use case "prioriser les secours"
- **Inconvénient** : On perd de l'info (agrégation des véhicules/usagers)

### Option B : 1 ligne par VÉHICULE  
- **Target** : Ce véhicule a-t-il un occupant décédé ?
- **Avantage** : On garde plus d'info sur le véhicule
- **Inconvénient** : Un même accident apparaît plusieurs fois

### Option C : 1 ligne par USAGER
- **Target** : Cet usager est-il décédé ?
- **Avantage** : On peut utiliser âge, sexe, équipement de chaque personne
- **Inconvénient** : Données non indépendantes (usagers du même accident sont corrélés)

In [10]:
# Comparer les 3 approches
print("=== Comparaison des 3 granularités ===")

# Option A : par accident
mortel_accident = usagers.groupby('Num_Acc')['grav'].apply(lambda x: (x == 2).any())
print(f"\nOption A - Par accident:")
print(f"  Lignes: {len(mortel_accident)}")
print(f"  Mortels: {mortel_accident.sum()} ({mortel_accident.mean()*100:.1f}%)")

# Option B : par véhicule
mortel_vehicule = usagers.groupby(['Num_Acc', 'id_vehicule'])['grav'].apply(lambda x: (x == 2).any())
print(f"\nOption B - Par véhicule:")
print(f"  Lignes: {len(mortel_vehicule)}")
print(f"  Mortels: {mortel_vehicule.sum()} ({mortel_vehicule.mean()*100:.1f}%)")

# Option C : par usager
print(f"\nOption C - Par usager:")
print(f"  Lignes: {len(usagers)}")
print(f"  Décédés: {(usagers['grav'] == 2).sum()} ({(usagers['grav'] == 2).mean()*100:.1f}%)")

=== Comparaison des 3 granularités ===

Option A - Par accident:
  Lignes: 54402
  Mortels: 3226 (5.9%)

Option B - Par véhicule:
  Lignes: 92654
  Mortels: 3258 (3.5%)

Option C - Par usager:
  Lignes: 125187
  Décédés: 3432 (2.7%)


---
## 4. Analyse de la target (gravité)

Comprendre la variable `grav` dans la table usagers.

In [11]:
grav_labels = {
    1: 'Indemne',
    2: 'Tué',
    3: 'Blessé hospitalisé',
    4: 'Blessé léger'
}

print("=== Distribution de la gravité (usagers) ===")
for val, label in grav_labels.items():
    count = (usagers['grav'] == val).sum()
    pct = count / len(usagers) * 100
    print(f"  {val} - {label}: {count:>6} ({pct:.1f}%)")

=== Distribution de la gravité (usagers) ===
  1 - Indemne:  52920 (42.3%)
  2 - Tué:   3432 (2.7%)
  3 - Blessé hospitalisé:  19126 (15.3%)
  4 - Blessé léger:  49709 (39.7%)


In [12]:
# Et si on faisait de la classification multi-classe ?
print("\n=== Pistes de modélisation ===")
print("\n1. Classification binaire : Mortel vs Non-mortel")
print("   → Ce qu'on fait actuellement")

print("\n2. Classification multi-classe : 4 niveaux de gravité")
print("   → Plus fin, mais plus difficile")

print("\n3. Classification binaire : Grave vs Non-grave")
grave = (usagers['grav'].isin([2, 3])).sum()
print(f"   → Grave (tué + hospitalisé) = {grave} ({grave/len(usagers)*100:.1f}%)")
print("   → Moins déséquilibré que mortel seul")


=== Pistes de modélisation ===

1. Classification binaire : Mortel vs Non-mortel
   → Ce qu'on fait actuellement

2. Classification multi-classe : 4 niveaux de gravité
   → Plus fin, mais plus difficile

3. Classification binaire : Grave vs Non-grave
   → Grave (tué + hospitalisé) = 22558 (18.0%)
   → Moins déséquilibré que mortel seul


---
## 5. Idées de feature engineering

Quelles nouvelles features pourrait-on créer ?

In [13]:
print("=== Idées de features à créer ===")

print("\n📍 LOCALISATION")
print("  - distance_hopital : distance au CHU le plus proche")
print("  - zone_rurale : basé sur densité population")
print("  - autoroute : catr == 1")

print("\n🚗 VÉHICULES")
print("  - ratio_poids : poids lourd vs véhicule léger")
print("  - vitesse_estimee : basée sur vma et type de route")
print("  - vehicule_vulnerable : vélo, moto, piéton")

print("\n👥 USAGERS")
print("  - age_conducteur : âge du conducteur principal")
print("  - nb_enfants : usagers de moins de 18 ans")
print("  - taux_ceinture : % d'usagers avec ceinture")

print("\n⏰ TEMPOREL")
print("  - nuit_weekend : combinaison heure + jour")
print("  - heure_pointe : 7-9h ou 17-19h")
print("  - vacances_scolaires : période à risque")

=== Idées de features à créer ===

📍 LOCALISATION
  - distance_hopital : distance au CHU le plus proche
  - zone_rurale : basé sur densité population
  - autoroute : catr == 1

🚗 VÉHICULES
  - ratio_poids : poids lourd vs véhicule léger
  - vitesse_estimee : basée sur vma et type de route
  - vehicule_vulnerable : vélo, moto, piéton

👥 USAGERS
  - age_conducteur : âge du conducteur principal
  - nb_enfants : usagers de moins de 18 ans
  - taux_ceinture : % d'usagers avec ceinture

⏰ TEMPOREL
  - nuit_weekend : combinaison heure + jour
  - heure_pointe : 7-9h ou 17-19h
  - vacances_scolaires : période à risque


---
## 🤔 Questions à se poser

Avant de continuer, réfléchis à :

1. **Quel est ton use case principal ?**
   - Priorisation des secours ? → granularité accident
   - Prévention routière ? → granularité usager
   - Assurance ? → granularité véhicule

2. **Quelles features sont disponibles en temps réel ?**
   - Au moment de l'appel au 15/18
   - vs après l'enquête

3. **Quel déséquilibre de classes est acceptable ?**
   - 5.9% mortels (très déséquilibré)
   - ~20% graves (plus équilibré)

4. **Quel modèle utiliser ?**
   - Random Forest (ce qu'on fait)
   - XGBoost / LightGBM (souvent meilleur)
   - Régression logistique (plus interprétable)